Benchmarks
----------

Import necessary libraries. We use ansitable to print a table of benchmarks and numpy for random state-space sampling.

In [21]:
import time
import panda_py
from panda_py import constants, motion
import numpy as np
import ansitable

Each function is evaluated for `num_samples` number of samples. Trajectories consist of `num_waypoints` waypoints each. We randomly sample the state-space defined by the joint limits for joint positions stored in `q_rand` and store the corresponding end-effector poses in `T_rand`.

In [22]:
num_samples = 10000
num_waypoints = 5
np.random.seed(0)
q_rand = np.random.uniform(low=constants.JOINT_LIMITS_LOWER,
                           high=constants.JOINT_LIMITS_UPPER,
                           size=(num_samples, 7))
T_rand = [panda_py.fk(q) for q in q_rand]

Execute and time the samples for inverse and forward kinematics as well joint and Cartesian trajectory generation. Total runtime is around 5 minutes for 10000 samples.

In [23]:
ik = []
for T in T_rand:
  t = time.time()
  panda_py.ik(T)
  ik.append(time.time() - t)

fk = []
for q in q_rand:
  t = time.time()
  panda_py.fk(q)
  fk.append(time.time() - t)

joint_motion = []
for i in range(num_samples):
  waypoints = []
  for k in range(num_waypoints):
    waypoints.append(q_rand[np.random.randint(0, num_samples - 1), :])
  t = time.time()
  try:
    motion.JointTrajectory(waypoints, timeout=1)
  except:
    continue
  joint_motion.append(time.time() - t)

cartesian_motion = []
for i in range(num_samples):
  waypoints = []
  for k in range(num_waypoints):
    waypoints.append(T_rand[np.random.randint(0, num_samples - 1)])
  t = time.time()
  try:
    motion.CartesianTrajectory(waypoints, timeout=1)
  except:
    continue
  cartesian_motion.append(time.time() - t)


Trajectory computation timed out after 1 seconds.
Trajectory computation timed out after 1 seconds.
Trajectory computation timed out after 1 seconds.
Trajectory computation timed out after 1 seconds.
Trajectory computation timed out after 1 seconds.


Pretty-print the resulting average runtimes.

In [24]:
def format_runtime(runtime):
    return format(np.average(runtime), '.2e')

table = ansitable.ANSITable('Function', 'Avg. Runtime (s)', border='thin')
table.row('fk', format_runtime(fk))
table.row('ik', format_runtime(ik))
table.row('JointMotion', format_runtime(joint_motion))
table.row('CartesianMotion', format_runtime(cartesian_motion))
table.print()

┌────────────────┬──────────────────┐
│       Function │ Avg. Runtime (s) │
├────────────────┼──────────────────┤
│             fk │         3.62e-06 │
│             ik │         3.07e-06 │
│    JointMotion │         1.83e-02 │
│CartesianMotion │         9.66e-03 │
└────────────────┴──────────────────┘

